In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/filelock-3.13.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2024.6.1-py3-none-any.whl.metadata (11 kB)
  Using cached https://download.pytorch.org/whl/setuptools-70.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached https://download.pytorch.org/whl/numpy-2.1.2-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached https://download.pytorch.org/whl/pillow-11.0.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached https://download.pytorch.org/whl

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
import torch.nn.functional as F

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])
])

train_data = datasets.ImageFolder(r'C:\Users\srisi\OneDrive\Desktop\sidxt\showcase\Fracture-Detection-Using-CNN\archive\dataset\train', transform=transform)
test_data = datasets.ImageFolder(r'C:\Users\srisi\OneDrive\Desktop\sidxt\showcase\Fracture-Detection-Using-CNN\archive\dataset\test', transform=transform)
val_data = datasets.ImageFolder(r'C:\Users\srisi\OneDrive\Desktop\sidxt\showcase\Fracture-Detection-Using-CNN\archive\dataset\val', transform=transform)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=True)


In [4]:
conv1 = nn.Conv2d(1, 32, 3)
conv2 = nn.Conv2d(32, 64, 3)
conv3 = nn.Conv2d(64, 128, 3)
fc1 = nn.Linear(128 * 26 * 26, 256)
fc2 = nn.Linear(256, 128)
fc3 = nn.Linear(128, 1)
pool = nn.MaxPool2d(2)
dropout = nn.Dropout(0.3)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
conv1.to(device)
conv2.to(device)
conv3.to(device)
fc1.to(device)
fc2.to(device)
fc3.to(device)
pool.to(device)
dropout.to(device)


def forward(x):
    x = pool(F.relu(conv1(x)))
    x = dropout(x)
    x = pool(F.relu(conv2(x)))
    x = dropout(x)
    x = pool(F.relu(conv3(x)))
    x = dropout(x)
    x = x.view(-1, 128 * 26 * 26)
    x = F.relu(fc1(x))
    x = dropout(x)
    x = F.relu(fc2(x))
    x = dropout(x)
    x = torch.sigmoid(fc3(x))
    return x
criterion = nn.BCELoss()
optimizer = optim.Adam(list(conv1.parameters()) + list(conv2.parameters()) + list(conv3.parameters()) + 
                       list(fc1.parameters()) + list(fc2.parameters()) + list(fc3.parameters()), lr=0.001)

In [5]:
for epoch in range(50):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = forward(inputs)
        loss = criterion(outputs, labels.unsqueeze(1).float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"epoch {epoch+1}, loss: {running_loss / len(train_loader)}")


epoch 1, loss: 0.6820088082043699
epoch 2, loss: 0.5619205885617308
epoch 3, loss: 0.46657308554926585
epoch 4, loss: 0.3333263346391131
epoch 5, loss: 0.22662419900875683
epoch 6, loss: 0.15737570406392562
epoch 7, loss: 0.09952742495942254
epoch 8, loss: 0.07862365273253488
epoch 9, loss: 0.05793126269407
epoch 10, loss: 0.04836756097921712
epoch 11, loss: 0.04275991820167144
epoch 12, loss: 0.039728412720061154
epoch 13, loss: 0.027671481167653
epoch 14, loss: 0.024861740189118946
epoch 15, loss: 0.02980133816261993
epoch 16, loss: 0.027255736198907993
epoch 17, loss: 0.022635604282087544
epoch 18, loss: 0.01450543705903843
epoch 19, loss: 0.022138212528621214
epoch 20, loss: 0.016083194159660325
epoch 21, loss: 0.013780462383389628
epoch 22, loss: 0.012048947284425637
epoch 23, loss: 0.02481416839156398
epoch 24, loss: 0.01108217476370231
epoch 25, loss: 0.007402780072976548
epoch 26, loss: 0.025481451389923183
epoch 27, loss: 0.06324032553692625
epoch 28, loss: 0.02134532778188096

In [6]:
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        #print("predicted - "+str(predicted))
        #print("label - "+str(labels))
        total += labels.size(0)
        correct += (predicted == labels.unsqueeze(1)).sum().item()

print(f"test accuracy: {100 * correct / total}%")

test accuracy: 99.49874686716792%


In [7]:
torch.save({
    'conv1_state_dict': conv1.state_dict(),
    'conv2_state_dict': conv2.state_dict(),
    'conv3_state_dict': conv3.state_dict(),
    'fc1_state_dict': fc1.state_dict(),
    'fc2_state_dict': fc2.state_dict(),
    'fc3_state_dict': fc3.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, 'main.pth')

In [8]:
%pip install scikit-learn

  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.15.2-cp312-cp312-win_amd64.whl (40.9 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- ----------------------------- 1/4 [scipy]
   ---------- -----------------

In [9]:
from sklearn.metrics import precision_recall_fscore_support
all_labels = []
all_predictions = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = forward(inputs)
        predicted = torch.round(outputs)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())
precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_predictions, average='binary')

print(f"Test Accuracy: {100 * correct / total}%")
print(f"Precision: {precision*100}")
print(f"Recall: {recall*100}")
print(f"F1 Score: {f1*100}")

Test Accuracy: 99.49874686716792%
Precision: 100.0
Recall: 99.0
F1 Score: 99.49748743718592
